In [1]:
import os
import sys
import json
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from typing import List, Tuple, Dict, Any, Optional

In [2]:
PATH_ROOT_DIR: str = ".."

INFO_TO_USE_FOR_CLUSTERING: List[str] = [
    'exception_message', 'essential_file_content']

PCA_COMPONENTS: int = 3

AI_MODEL: str = "llama-3.1-70b-versatile"  # or "gpt-4o-mini"

N_POSITIVE_EXAMPLES_TO_SHOW_LLM: int = 5

# Test run
# DIR_PROGRAM_BANK = os.path.join("..", "program_bank", "v001", "2024_10_04__14_48__qiskit")
# Run with 10k programs: v001.yaml
# program_bank/v001/2024_10_04__15_10__qiskit
# DIR_PROGRAM_BANK = os.path.join("..", "program_bank", "v001", "2024_10_04__15_10__qiskit")
# program_bank/v002/2024_10_07__17_03__qiskit
# DIR_PROGRAM_BANK: str = os.path.join(
#     "..", "program_bank", "v004", "2024_10_18__16_09__qiskit")

# DIR_SAMPLE_ERRORS: str = os.path.join(
#     "..", "program_bank", "v004", "2024_10_18__16_09__sample_errors")


# DIR_PROGRAM_BANK: str = os.path.join(
#     "..", "program_bank", "v005", "2024_10_22__14_17__qiskit")
# DIR_SAMPLE_ERRORS: str = os.path.join(
#     "..", "program_bank", "v005", "2024_10_22__14_17__sample_errors")

# inspected folders
# program_bank/v005/2024_10_22__22_32__qiskit
# program_bank/v007/2024_10_31__17_51__qiskit
# program_bank/v008/2024_11_07__15_05__qiskit
# program_bank/v009/2024_11_20__13_42__qiskit
DIR_PROGRAM_BANK: str = os.path.join(
    "..", "program_bank", "v009", "2024_11_20__13_42__qiskit")
DIR_SAMPLE_ERRORS: str = os.path.join(
    "..", "program_bank", "v009",
    "2024_11_20__13_42__sample_errors_no_pennylane")


PROMPT_CONTENT: str = "BOTH_ERROR_AND_PROGRAM"

In [3]:
# print the current folder
print(f"Current folder: {os.getcwd()}")

all_global_vars = dict(globals().copy())
# print all global variables
for k, v in all_global_vars.items():
    if k.isupper():
        print(f"{k}: {v}")

Current folder: /home/paltenmo/projects/crossplatform/notebooks
PATH_ROOT_DIR: ..
INFO_TO_USE_FOR_CLUSTERING: ['exception_message', 'essential_file_content']
PCA_COMPONENTS: 3
AI_MODEL: llama-3.1-70b-versatile
N_POSITIVE_EXAMPLES_TO_SHOW_LLM: 5
DIR_PROGRAM_BANK: ../program_bank/v009/2024_11_20__13_42__qiskit
DIR_SAMPLE_ERRORS: ../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane
PROMPT_CONTENT: BOTH_ERROR_AND_PROGRAM


In [4]:
import openai
import groq
PATH_GROQ_TOKEN = os.path.join(PATH_ROOT_DIR, "groq_token.txt")
GROQ_TOKEN = Path(PATH_GROQ_TOKEN).read_text().strip()

# Export the GROQ_TOKEN as GROQ_API_KEY
os.environ['GROQ_API_KEY'] = GROQ_TOKEN

PATH_OPENAI_API_KEY = os.path.join(PATH_ROOT_DIR, "openai_token.txt")
OPENAI_API_KEY = Path(PATH_OPENAI_API_KEY).read_text().strip()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [5]:
# count generated programs in the program bank
count_generated_programs = len(list(Path(DIR_PROGRAM_BANK).rglob("*.py")))
print(f"Number of generated programs: {count_generated_programs}")

# count the number of errors
count_errors = len(list(Path(DIR_PROGRAM_BANK).rglob("*_error.json")))
print(f"Number of errors: {count_errors}")

Number of generated programs: 7168
Number of errors: 14818


## Read Files

In [25]:
from multiprocessing import Pool
import re


def read_json_file(file_path: str) -> Dict[str, Any]:
    with open(file_path, 'r') as file:
        data = json.load(file)
        data["error_file"] = os.path.basename(file_path)
        return data


def read_all_programs(dir_path: str) -> pd.DataFrame:
    json_files = [os.path.join(dir_path, f) for f in os.listdir(
        dir_path) if re.match(r'.*_error\.json$', f)]
    records = []

    with Pool() as pool:
        results = pool.map(read_json_file, json_files)
        records.extend(results)

    return pd.DataFrame(records)


# Use the function to read all programs
df_errors = read_all_programs(DIR_PROGRAM_BANK)

# augment the dataframe with the proram content


def read_file_content(file_name: str) -> str:
    file_path = os.path.join(DIR_PROGRAM_BANK, file_name)
    with open(file_path, 'r') as file:
        return file.read()


# Use parallel apply to read the file content
with Pool() as pool:
    df_errors['file_content'] = pool.map(
        read_file_content, df_errors['current_file'])

# Alternatively, you can use apply without parallel processing
# df_errors['file_content'] = df_errors['current_file'].apply(read_file_content)

df_errors.head()
n_errors = len(df_errors)
print(f"Number of errors: {n_errors}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Number of errors: 14818


# Exclude Pennylane (errors)

In [9]:
df_errors['involved_functions'] = df_errors['involved_functions'].astype(
    str)
df_errors = df_errors[
    ~df_errors['involved_functions'].str.contains('pennylane')]
n_errors = len(df_errors)
print(f"Number of errors without PennyLane: {n_errors}")

Number of errors without PennyLane: 8719


# Minimize Sample

In [ ]:
import shutil

# keep only errors with `The circuits are not equivalent` in the exception message
df_errors = df_errors[df_errors['exception_message'].str.contains(
    'The circuits are not equivalent')]


SAMPLE_SIZE = 10
actual_sample_size = min(SAMPLE_SIZE, len(df_errors))


# pick a sample of error files
sample_errors = df_errors.sample(actual_sample_size, random_state=42)

# get the paths of the error files
sample_errors_paths = [os.path.join(
    DIR_PROGRAM_BANK, f) for f in sample_errors['error_file']]


def get_corresponding_program_file(path_error_file: str) -> str:
    # remove the fixed length prefix "_ba6b02_error.json"
    return path_error_file[:-18] + ".py"


def copy_errors_to_sample_folder(
        sample_errors_paths: List[str],
        dir_sample_errors: str):
    os.makedirs(dir_sample_errors, exist_ok=True)
    for file_path in sample_errors_paths:
        shutil.copy(file_path, dir_sample_errors)
        print(f"Copying {file_path} to {dir_sample_errors}")
        corresponding_program_file = get_corresponding_program_file(file_path)
        shutil.copy(corresponding_program_file, dir_sample_errors)
        print(f"Copying {corresponding_program_file} to {dir_sample_errors}")


copy_errors_to_sample_folder(sample_errors_paths, DIR_SAMPLE_ERRORS)

Copying ../program_bank/v009/2024_11_20__13_42__qiskit/qiskit_circuit_5q_10g_1511_600fe6_0353c4_error.json to ../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane
Copying ../program_bank/v009/2024_11_20__13_42__qiskit/qiskit_circuit_5q_10g_1511_600fe6.py to ../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane
Copying ../program_bank/v009/2024_11_20__13_42__qiskit/qiskit_circuit_5q_10g_5923_1fb05d_394db9_error.json to ../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane
Copying ../program_bank/v009/2024_11_20__13_42__qiskit/qiskit_circuit_5q_10g_5923_1fb05d.py to ../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane
Copying ../program_bank/v009/2024_11_20__13_42__qiskit/qiskit_circuit_5q_10g_5585_30abba_76b749_error.json to ../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane
Copying ../program_bank/v009/2024_11_20__13_42__qiskit/qiskit_circuit_5q_10g_5585_30abba.py to ../program_bank/v009/2024_11_20__13_42__sample_err

In [11]:
# get the content of their error messages
sample_errors['exception_message']

12723    The circuits are not equivalent: /workspace/qi...
693      The circuits are not equivalent: /workspace/qi...
10624    The circuits are not equivalent: /workspace/qi...
3913     The circuits are not equivalent: /workspace/qi...
5149     The circuits are not equivalent: /workspace/qi...
11150    The circuits are not equivalent: /workspace/qi...
842      The circuits are not equivalent: /workspace/qi...
10721    The circuits are not equivalent: /workspace/qi...
11223    The circuits are not equivalent: /workspace/qi...
11601    The circuits are not equivalent: /workspace/qi...
Name: exception_message, dtype: object

In [30]:
import sys
sys.path.append("..")

## Triage All Files 
Including:
1. minimization
2. rerunning the code
3. running the analysis notebook

In [13]:
%load_ext autoreload
%autoreload 2

In [15]:
from analysis_and_reporting.triage import analyze_and_report

error_paths_in_sample_folder = [
    os.path.join(DIR_SAMPLE_ERRORS, f)
    for f in os.listdir(DIR_SAMPLE_ERRORS)
    if re.match(r'.*_error\.json$', f)
]

n_error_files = len(error_paths_in_sample_folder)
print(f"Error paths in sample folder ({n_error_files}):")
for error_path in error_paths_in_sample_folder:
    print(error_path)

# python -m analysis_and_reporting.triage --path_error_json program_bank/v007/2024_10_31__17_51__qiskit/qiskit_circuit_5q_10g_9_160f04_cfe9f1_error.json --parent_report_folder reports/v008 --clue_message 'not equivalent'


# def analyze_and_report(
#         path_error_json: str, path_program: str, parent_report_folder: str,
#         clue_message: str, analysis_notebook: str)

for error_path in error_paths_in_sample_folder:
    try:
        analyze_and_report(
            path_error_json=error_path,
            path_program=get_corresponding_program_file(error_path),
            parent_report_folder=os.path.join("..", "reports", "v009"),
            clue_message='not equivalent',
            analysis_notebook='010_Triage_Not_Equivalence.ipynb'
        )
    except Exception as e:
        print(f"Error: {e}")
        print("Skipping to the next error file")

Error paths in sample folder (10):
../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane/qiskit_circuit_5q_10g_4342_faa083_a812bf_error.json
../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane/qiskit_circuit_5q_10g_6316_146402_6a36f0_error.json
../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane/qiskit_circuit_5q_10g_6533_f0b204_dbc69c_error.json
../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane/qiskit_circuit_5q_10g_4756_28bbeb_444f6d_error.json
../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane/qiskit_circuit_5q_10g_144_2ca34b_7b044a_error.json
../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane/qiskit_circuit_5q_10g_680_6d896d_1fd766_error.json
../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane/qiskit_circuit_5q_10g_5399_c4b732_f59bef_error.json
../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane/qiskit_circuit_5q_10g_609_0c26d1_ee02f4_error.json
../program_bank/

[14:49:51] Created report folder: ../reports/v009/2024_11_21__14_49                                   ]8;id=600267;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py\triage.py]8;;\:]8;id=821025;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py#131\131]8;;\

           Copied files to: ../reports/v009/2024_11_21__14_49                                         ]8;id=566959;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py\triage.py]8;;\:]8;id=440829;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py#140\140]8;;\

[14:49:51] Involved functions:                                                             ]8;id=234626;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=101226;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#206\206]8;;\

             - compare_qasm_via_qcec                                                       ]8;id=317805;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=556817;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

...

[15:27:12] Minimized file saved as:                                                        ]8;id=250509;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=510256;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#240\240]8;;\
           ../reports/v009/2024_11_21__15_23/qiskit_circuit_5q_10g_2997_458c2e_f885b3_erro                         
           r_min.py                                                                                                

Output: Saved the Pytket circuit to /workspace/qiskit_circuit_5q_10g_2997_458c2e_f885b3_error_min_qc_pytket.qasm
Saved the Qiskit circuit to /workspace/qiskit_circuit_5q_10g_2997_458c2e_f885b3_error_min_qc_qiskit.qasm
Exported QASM files: [('/workspace/qiskit_circuit_5q_10g_2997_458c2e_f885b3_error_min_qc_pytket.qasm', 'export_to_qasm_with_pytket'), ('/workspace/qiskit_circuit_5q_10g_2997_458c2e_f885b3_error_min_qc_qiskit.qasm', 'export_to_qasm_with_qiskit')]
Log has been saved to /workspace/qiskit_circuit_5q_10g_2997_458c2e_f885b3_error_min_da3a45_error.json



[15:27:18] Program qiskit_circuit_5q_10g_2997_458c2e_f885b3_error_min.py executed              ]8;id=134878;file:///home/paltenmo/projects/crossplatform/generators/docker_tooling.py\docker_tooling.py]8;;\:]8;id=151265;file:///home/paltenmo/projects/crossplatform/generators/docker_tooling.py#46\46]8;;\
           successfully.                                                                                           

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

[15:27:21] Analysis notebook executed: ../reports/v009/2024_11_21__15_23/analysis_output.ipynb        ]8;id=463065;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py\triage.py]8;;\:]8;id=328971;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/triage.py#163\163]8;;\

# Inspect Crash Errors

In [7]:
# ! pip install datasets

In [8]:
df_errors.head()

,exception_message,stack_trace,current_file,involved_functions,error_file,file_content
0,The circuits are not equivalent: /workspace/qi...,"Traceback (most recent call last):\n File ""/w...",qiskit_circuit_5q_10g_2296_757d63.py,"[compare_qasm_via_qcec, export_to_qasm_with_pe...",qiskit_circuit_5q_10g_2296_757d63_aadf90_error...,# Section: Prologue\nfrom qiskit import Quantu...
1,The circuits are not equivalent: /workspace/qi...,"Traceback (most recent call last):\n File ""/w...",qiskit_circuit_5q_10g_6206_4d872f.py,"[compare_qasm_via_qcec, export_to_qasm_with_pe...",qiskit_circuit_5q_10g_6206_4d872f_bb556c_error...,# Section: Prologue\nfrom qiskit import Quantu...
2,DD for gatemeasure not available!,"Traceback (most recent call last):\n File ""/w...",qiskit_circuit_5q_10g_7114_290fb8.py,"[compare_qasm_via_qcec, export_to_qasm_with_py...",qiskit_circuit_5q_10g_7114_290fb8_ac0452_error...,# Section: Prologue\nfrom qiskit import Quantu...
3,The circuits are not equivalent: /workspace/qi...,"Traceback (most recent call last):\n File ""/w...",qiskit_circuit_5q_10g_224_e4cbeb.py,"[compare_qasm_via_qcec, export_to_qasm_with_bq...",qiskit_circuit_5q_10g_224_e4cbeb_7b2138_error....,# Section: Prologue\nfrom qiskit import Quantu...
4,Could not import second circuit: [addClassical...,"Traceback (most recent call last):\n File ""/w...",qiskit_circuit_5q_10g_2276_57cb36.py,"[compare_qasm_via_qcec, export_to_qasm_with_pe...",qiskit_circuit_5q_10g_2276_57cb36_598cfe_error...,# Section: Prologue\nfrom qiskit import Quantu...


In [ ]:
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModel
import torch
from sentence_transformers import SentenceTransformer

# create a dataset from the error dataframe
dataset = Dataset.from_pandas(df_errors)

# inspect the dataset
dataset

# use deepseek-ai/DeepSeek-Coder-V2-Lite-Base
# from datasets import Dataset
# dataset = Dataset.from_dict({"a": [0, 1, 2]})
# dataset.map(lambda batch: {"b": batch["a"] * 2}, batched=True)  # new column with 6 elements: [0, 1, 2, 0, 1, 2]
# Load the tokenizer and model
# EMBEDDING_MODEL = "TinyLlama/TinyLlama_v1.1_math_code"
# EMBEDDING_MODEL = "all-MiniLM-L6-v2"
EMBEDDING_MODEL = "multi-qa-mpnet-base-cos-v1"
# EMBEDDING_MODEL = "deepseek-ai/DeepSeek-Coder-V2-Lite-Base"
# Load a pretrained Sentence Transformer model
model = SentenceTransformer(EMBEDDING_MODEL)

# Calculate embeddings for the exception_message column
df_errors['exception_message_embeddings'] = list(
    model.encode(
        df_errors['exception_message'].tolist(),
        batch_size=32, show_progress_bar=True))

# Inspect the embeddings
df_errors['exception_message_embeddings'].head()

/home/paltenmo/.conda/envs/crosspl/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Batches:   0%|          | 0/464 [00:00<?, ?it/s]

0    [-0.014987822, -0.058666665, 0.0065132664, 0.0...
1    [-0.013780621, -0.057648543, 0.0038838007, 0.0...
2    [-0.038875815, 0.023484863, -0.01293374, -0.04...
3    [-0.012693973, -0.064225405, -0.0025399348, 0....
4    [-0.042616934, 0.035096787, 0.025242256, -0.00...
Name: exception_message_embeddings, dtype: object

In [10]:
# ! pip install fpsample

In [11]:
import fpsample
import numpy as np

# Extract the embeddings from the dataframe
embeddings = np.array(df_errors['exception_message_embeddings'].tolist())

# Sample 1024 points using various FPS methods

# # Vanilla FPS
fps_samples_idx = fpsample.fps_sampling(embeddings, 1024)

# Get the sampled data
sampled_data = df_errors.iloc[fps_samples_idx]
sampled_data.head()

,exception_message,stack_trace,current_file,involved_functions,error_file,file_content,exception_message_embeddings
2702,Cannot parse gate of type: ryy\nLine:12.,"Traceback (most recent call last):\n File ""/u...",qiskit_circuit_5q_10g_902_d9442f.py,"[import_from_qasm_with_pytket, export_to_qasm_...",qiskit_circuit_5q_10g_902_d9442f_6cc811_error....,# Section: Prologue\nfrom qiskit import Quantu...,"[0.008345557, 0.009281964, 0.013390886, 0.0286..."
14319,map::at,"Traceback (most recent call last):\n File ""/w...",qiskit_circuit_5q_10g_1066_e89a16.py,"[compare_qasm_via_qcec, export_to_qasm_with_py...",qiskit_circuit_5q_10g_1066_e89a16_e7d7b8_error...,# Section: Prologue\nfrom qiskit import Quantu...,"[-0.008285877, -0.10523057, -0.018257696, 0.02..."
14479,DD for non-unitary operation not available!,"Traceback (most recent call last):\n File ""/w...",qiskit_circuit_5q_10g_5928_6929be.py,"[compare_qasm_via_qcec, export_to_qasm_with_py...",qiskit_circuit_5q_10g_5928_6929be_617385_error...,# Section: Prologue\nfrom qiskit import Quantu...,"[-0.019316358, 0.01631485, 0.010183435, -0.083..."
14351,Operation MidMeasureMP not supported by the QA...,"Traceback (most recent call last):\n File ""/u...",qiskit_circuit_5q_10g_6431_6ccf80.py,[export_to_qasm_with_pennylane],qiskit_circuit_5q_10g_6431_6ccf80_cd55cd_error...,# Section: Prologue\nfrom qiskit import Quantu...,"[-0.012096606, -0.031633537, 0.000603492, -0.0..."
14785,Classical register redeclared: reg_1_c.,"Traceback (most recent call last):\n File ""/w...",qiskit_circuit_5q_10g_1606_ad0979.py,"[import_from_qasm_with_bqskit, export_to_qasm_...",qiskit_circuit_5q_10g_1606_ad0979_f464cf_error...,# Section: Prologue\nfrom qiskit import Quantu...,"[0.03173648, 0.070124455, -0.01046665, 0.00547..."


In [ ]:
pd.set_option('display.max_colwidth', None)
i = 2
size_chunk = 30
sampled_data[["exception_message", "error_file", "current_file"]
             ].iloc[i*size_chunk:(i+1)*size_chunk]

,exception_message,error_file,current_file
13394,Could not import first circuit: <input>:7:1:\nIdentifier 'reg_1_c' already declared.,qiskit_circuit_5q_10g_3316_ba1a73_a932bf_error.json,qiskit_circuit_5q_10g_3316_ba1a73.py
2704,"""<input>:873,5: 'reg_creg' is already defined""",qiskit_circuit_5q_10g_6836_ec095d_26cfcb_error.json,qiskit_circuit_5q_10g_6836_ec095d.py
6563,"""<input>:12,5: 'reg_1_c' is already defined""",qiskit_circuit_5q_10g_537_b1825c_e88272_error.json,qiskit_circuit_5q_10g_537_b1825c.py
1016,"The circuits are not equivalent: /workspace/qiskit_circuit_5q_10g_4426_998773_qc_bqskit.qasm, /workspace/qiskit_circuit_5q_10g_4426_998773_qc_pennylane.qasm",qiskit_circuit_5q_10g_4426_998773_d98e50_error.json,qiskit_circuit_5q_10g_4426_998773.py
8261,Could not import first circuit: <input>:14:1:\nIdentifier 'reg_2_c0' already declared.,qiskit_circuit_5q_10g_4368_43ca5b_3bf985_error.json,qiskit_circuit_5q_10g_4368_43ca5b.py
3261,"The circuits are not equivalent: /workspace/qiskit_circuit_5q_10g_5089_adc03e_qc_pytket.qasm, /workspace/qiskit_circuit_5q_10g_5089_adc03e_qc_qiskit.qasm",qiskit_circuit_5q_10g_5089_adc03e_5ab79a_error.json,qiskit_circuit_5q_10g_5089_adc03e.py
14707,Could not import first circuit: <input>:8:1:\nIdentifier 'reg_1_c0' already declared.,qiskit_circuit_5q_10g_6592_46e100_4421c3_error.json,qiskit_circuit_5q_10g_6592_46e100.py
13360,"The circuits are not equivalent: /workspace/qiskit_circuit_5q_10g_233_8c1b4d_qc_bqskit.qasm, /workspace/qiskit_circuit_5q_10g_233_8c1b4d_qc_qiskit.qasm",qiskit_circuit_5q_10g_233_8c1b4d_c4f252_error.json,qiskit_circuit_5q_10g_233_8c1b4d.py
14534,"""<input>:9,0: 'ryy' is not defined in this scope""",qiskit_circuit_5q_10g_5600_cd4a39_fddc8b_error.json,qiskit_circuit_5q_10g_5600_cd4a39.py
10794,"The circuits are not equivalent: /workspace/qiskit_circuit_5q_10g_2441_dbdbec_qc_pennylane.qasm, /workspace/qiskit_circuit_5q_10g_2441_dbdbec_qc_pytket.qasm",qiskit_circuit_5q_10g_2441_dbdbec_ba2dd7_error.json,qiskit_circuit_5q_10g_2441_dbdbec.py


In [ ]:
# relevant errors
relevant_error_files_with_message = [
    # probably QCEC error
    ("qiskit_circuit_5q_10g_2382_eaac27_73fae5_error.json",
     "classic-controlled operation is not a StandardOperation"),
    # interesting pair of divergent platforms
    ("qiskit_circuit_5q_10g_5557_dc52a0_027c07_error.json",
     "The circuits are not equivalent")
]

In [ ]:
error_paths = [os.path.join(DIR_PROGRAM_BANK, f)
               for f, _ in relevant_error_files_with_message]

# copy the relevant errors to the sample folder
copy_errors_to_sample_folder(error_paths, DIR_SAMPLE_ERRORS)

Copying ../program_bank/v009/2024_11_20__13_42__qiskit/qiskit_circuit_5q_10g_2382_eaac27_73fae5_error.json to ../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane
Copying ../program_bank/v009/2024_11_20__13_42__qiskit/qiskit_circuit_5q_10g_2382_eaac27.py to ../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane
Copying ../program_bank/v009/2024_11_20__13_42__qiskit/qiskit_circuit_5q_10g_5557_dc52a0_027c07_error.json to ../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane
Copying ../program_bank/v009/2024_11_20__13_42__qiskit/qiskit_circuit_5q_10g_5557_dc52a0.py to ../program_bank/v009/2024_11_20__13_42__sample_errors_no_pennylane


In [ ]:
from analysis_and_reporting.triage import analyze_and_report

# Extract the error paths from the relevant list
error_paths_in_sample_folder = [
    os.path.join(DIR_SAMPLE_ERRORS, error_file)
    for error_file, clue in relevant_error_files_with_message
]

n_error_files = len(error_paths_in_sample_folder)
print(f"Error paths in sample folder ({n_error_files}):")
for error_path in error_paths_in_sample_folder:
    print(error_path)

# Analyze and report for each relevant error file
for error_path, clue_message in relevant_error_files_with_message:
    try:
        analyze_and_report(
            path_error_json=os.path.join(DIR_SAMPLE_ERRORS, error_path),
            path_program=get_corresponding_program_file(
                os.path.join(DIR_SAMPLE_ERRORS, error_path)),
            parent_report_folder=os.path.join("..", "reports", "v009"),
            clue_message=clue_message,
            analysis_notebook='010_Triage_Not_Equivalence.ipynb')
    except Exception as e:
        print(f"Error: {e}")
        print("Skipping to the next error file")

In [21]:
file_name = "qiskit_circuit_5q_10g_2382_eaac27_73fae5_error.json"
# file_name = "qiskit_circuit_5q_10g_2382_eaac27.py"
file_path = os.path.join(DIR_PROGRAM_BANK, file_name)
print(file_path)

with open(file_path, 'r') as file:
    file_content = file.read()

print(file_content)

../program_bank/v009/2024_11_20__13_42__qiskit/qiskit_circuit_5q_10g_2382_eaac27_73fae5_error.json
{
    "exception_message": "Underlying operation of classic-controlled operation is not a StandardOperation.\n   |\u001b   |\u001b   |\u001b   |\u001b   |\u001b   |\u001b  c[0] == 1",
    "stack_trace": "Traceback (most recent call last):\n  File \"/workspace/qiskit_circuit_5q_10g_2382_eaac27.py\", line 566, in <module>\n    compare_call(a_file, b_file)\n  File \"/workspace/qiskit_circuit_5q_10g_2382_eaac27.py\", line 503, in compare_qasm_via_qcec\n    result = qcec.verify(\n  File \"/usr/local/lib/python3.10/site-packages/mqt/qcec/verify.py\", line 56, in verify\n    ecm = EquivalenceCheckingManager(circ1, circ2, configuration)\nValueError: Underlying operation of classic-controlled operation is not a StandardOperation.\n   |\u001b   |\u001b   |\u001b   |\u001b   |\u001b   |\u001b  c[0] == 1\n",
    "current_file": "qiskit_circuit_5q_10g_2382_eaac27.py",
    "involved_functions": [
     